In [1]:
{-# LANGUAGE LambdaCase #-}

import qualified Data.Graph          as G
import qualified Data.Array          as A
import qualified Data.Vector.Mutable as V

import Prelude hiding (lookup)

import Data.Traversable (sequenceA)
import Control.Monad.ST
import Data.STRef
import Control.Monad (forM_, when)
import Data.Maybe (isNothing, fromJust, isJust)

import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as BC

denormalise = subtract
normalise   = (+)
other n v = 2*n - v
clauses n [u,v] = [(other n u, v), (other n v, u)]

tarjan :: Int -> G.Graph -> [Maybe [Int]]
tarjan n graph = runST $ do
    index    <- newSTRef (0 :: Int)
    stack    <- newSTRef []
    stackSet <- V.replicate size False
    indices  <- V.replicate size Nothing
    lowlinks <- V.replicate size Nothing
    output   <- newSTRef []

    forM_ (G.vertices graph) $ \v -> do
        vIndex <- flip (V.read) v indices
        when (isNothing vIndex) $
            strongConnect n v graph index stack stackSet indices lowlinks output

    readSTRef output
    where
        size = snd (A.bounds graph) + 1


strongConnect
    :: Int
    -> Int
    -> G.Graph
    -> STRef s Int
    -> STRef s [Int]
    -> V.STVector s Bool
    -> V.STVector s (Maybe Int)
    -> V.STVector s (Maybe Int)
    -> STRef s [Maybe [Int]]
    -> ST    s ()
strongConnect n v graph index stack stackSet indices lowlinks output = do
    i <- readSTRef index
    insert v i indices
    insert v i lowlinks
    increment index
    push stack stackSet v

    forM_ (graph A.! v) $ \w -> lookup w indices >>= \case
        Nothing     -> do
            strongConnect n w graph index stack stackSet indices lowlinks output
            vLowLink <- fromJust <$> lookup v lowlinks
            wLowLink <- fromJust <$> lookup w lowlinks
            insert v (min vLowLink wLowLink) lowlinks
        Just wIndex -> do
            wOnStack <- flip (V.read) w stackSet
            when wOnStack $ do
                vLowLink <- fromJust <$> lookup v lowlinks
                insert v (min vLowLink wIndex) lowlinks

    vLowLink <- fromJust <$> lookup v lowlinks
    vIndex   <- fromJust <$> lookup v indices
    when (vLowLink == vIndex) $ do
        scc <- addSCC n v (Just []) stack stackSet
        modifySTRef' output (scc:)
    where
        lookup value hashMap     = flip (V.read) value hashMap
        insert key value hashMap = V.write hashMap key (Just value)
        increment counter        = modifySTRef' counter (+1)

addSCC :: Int -> Int -> Maybe [Int] -> STRef s [Int] -> V.STVector s Bool -> ST s (Maybe [Int])
addSCC n v (Just scc) stack stackSet = do
    w <- pop stack stackSet
    if (other n w) `elem` scc
        then return Nothing
        else
            let scc' = Just (w:scc)
            in if w == v 
                then return scc'
                else addSCC n v scc' stack stackSet

push :: STRef s [Int] -> V.STVector s Bool -> Int -> ST s ()
push stack stackSet e = do
    modifySTRef' stack    (e:)
    V.write stackSet e True

pop :: STRef s [Int] -> V.STVector s Bool -> ST s Int
pop stack stackSet = do
    e <- head <$> readSTRef stack
    modifySTRef' stack tail
    V.write stackSet e False
    return e

In [2]:
checkSat name = do
    p <- map (map (fst . fromJust . BC.readInt) . BC.words) . BC.lines <$> B.readFile name
    let pNo    = head $ head p
        pn     = map (map (normalise pNo)) $ tail p
        pGraph = G.buildG (0,2*pNo) $ concatMap (clauses pNo) pn
    return . isJust . sequenceA $ tarjan pNo pGraph

In [3]:
checkSat "2sat1.txt"

True

In [4]:
checkSat "2sat2.txt"

False

In [5]:
checkSat "2sat3.txt"

True

In [6]:
checkSat "2sat4.txt"

True

In [7]:
checkSat "2sat5.txt"

False

In [8]:
checkSat "2sat6.txt"

False